# Resolução Desafio da Semana 1

In [2]:
import requests
from bs4 import BeautifulSoup

def soup_object(url):
  response = requests.get(url)
  return BeautifulSoup(response.text, 'html.parser')

def get_total_pages():
  soup = soup_object('https://books.toscrape.com/catalogue/page-1.html')

  pages_item = soup.find('li', class_='current').text
  total_pages = pages_item.split()[-1]

  return int(total_pages)

def fetch_books(page_number):
  url = f'https://books.toscrape.com/catalogue/page-{page_number}.html'

  soup = soup_object(url)

  books = []

  book_items = soup.find_all('article', class_='product_pod')

  for book_item in book_items:
    book_title = book_item.find('h3').find('a').text
    book_price = book_item.find('p', class_='price_color').text
    book_image = book_item.find('img').get('src')
    # EXTRA: book_image vai retornar o coaminho da imagem com: '../media/cache/outra/coisa/aqui/a_imagem.jpg', mas eu nao consigo acessar essa imagem
    # com esse caminho, porem posso adicionar o nome do site no lugar do '..', ficando assim:
    book_image_url = 'https://books.toscrape.com' + book_image.replace('..', '') # substituo .. por nada/branco e antes dele coloco o site
    book_stock = book_item.find('p', class_='instock availability').text.strip()
    book_link = book_item.find('h3').find('a').get('href')
    book_rating = book_item.find('p').get('class')[1]


    books.append({'titulo': book_title, 'preço': book_price, 'imagem': book_image, 'imagem caminho completo': book_image_url, 'estoque': book_stock, 'link': book_link, 'avaliação': book_rating})


  return books

def main():
  total_pages = get_total_pages()

  for page_number in range(1, total_pages + 1):
    print(f'Página {page_number}')
    print(fetch_books(page_number))


main()

Página 1
[{'titulo': 'A Light in the ...', 'preço': 'Â£51.77', 'imagem': '../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg', 'imagem caminho completo': 'https://books.toscrape.com/media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg', 'estoque': 'In stock', 'link': 'a-light-in-the-attic_1000/index.html', 'avaliação': 'Three'}, {'titulo': 'Tipping the Velvet', 'preço': 'Â£53.74', 'imagem': '../media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg', 'imagem caminho completo': 'https://books.toscrape.com/media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg', 'estoque': 'In stock', 'link': 'tipping-the-velvet_999/index.html', 'avaliação': 'One'}, {'titulo': 'Soumission', 'preço': 'Â£50.10', 'imagem': '../media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg', 'imagem caminho completo': 'https://books.toscrape.com/media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg', 'estoque': 'In stock', 'link': 'soumission_998/index.html', 'avaliação': 'One'}, {'titulo': 'Sharp Objects'

# Desafio

Busque no site: https://cultura.jundiai.sp.gov.br/agenda-cultural/ todos os evento que estão para acontecer em Jundiaí e forrmate os dados para que fiquem como no exemplo abaixo:

Evento: {Titulo do evento}

Local: {Local do evento}

Data: {Somente a data do evento: dd/mm/YYYY}
# .


**Extra**: Calcular e adicionar a quantidade de dias que faltam para o evento contando a partir da data de hoje (Obs.: Será preciso importar a biblioteca datetime), ficando assim:

Evento: {Titulo do evento}

Local: {Local do evento}

Data: {Somente a data do evento: dd/mm/YYYY}

Faltam {x} dias para o evento

In [7]:
import requests
import datetime as dt
from bs4 import BeautifulSoup

EVENTS = {}

# Aqui o numero total de paginas
for i in range(1, 10):
  if (i == 1): # Como a url é diferente da primeira pagina para as demais, faço esse truque
      URL = 'https://cultura.jundiai.sp.gov.br/agenda-cultural/'
  else:
      URL = f'https://cultura.jundiai.sp.gov.br/agenda-cultural/page/{i}/'

  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')

  news_list = soup.find_all('div', class_='noticia-lista') # Quero todos os itens que estao detro da tag DIV e class = 'noticia-lista'

  for item in news_list: # percorro todos os itens e pego as informações que quero
        title = item.find('div', class_='titulo-lista').text
        local = item.find('div', class_='resumo-lista').text
        img = item.find('img', class_='wp-post-image').get('src')


        date = item.span.p.text # outra forma de buscar a informação sem a função .find()
        date_split = date.strip().split(' ', maxsplit=2) # strip() remove os espaços em branco da direita e esquerda
                                                  # maxplit=2 separa somente em os dois o texto onde encontra espaço em brando e o resto mantem se mexer
                                                  # pode testar com print(date_split)
        date = date_split[1]  # a data está na posiçao 2 do array

        EVENTS[title] = {'local': local, 'date': date} # é dicionario dentro de um dicionario, entao por exemplo ele ficará assim:
                                                      # {'O titulo do espetaculo': {'local': 'Fatec', 'date': '13/09/2024'}}


        for event in EVENTS:# Percorro todos os eventos, event traz a chave/key com o 'O titulo do espetaculo'
          local = EVENTS[event]['local'] # entao com a chave consigo pegar o local e a data
          date = EVENTS[event]['date']

          today = dt.date.today()  # pego o dia de hoje no formato: dia/mes/ano

          event_date = dt.datetime.strptime(date, '%d/%m/%Y').date() # formato a data do evento para o formato: dia/mes/ano

          date_difference = event_date - today # Pego a diferença entre a data de hoje e a do evento

          print(f'Evento: {event} \nLocal: {local} \nData: {date} \nFaltam {date_difference.days} dias para o evento', end='\n\n')  # outra forma (com f-string) de imprimir os valores na tela
          # mais informaçoes: https://pythonacademy.com.br/blog/f-strings-no-python



Streaming output truncated to the last 5000 lines.
Evento: ‚ÄúEsta√ß√£o Clown‚Äù, com Cau Silva 
Local: F√°brica das Inf√¢ncias Japy 
Data: 29/09/2024 
Faltam 15 dias para o evento

Evento: 8¬∫ FESTEJU – ‚ÄúEntre Mundos‚Äù, com a Cia. Pulo do Gato 
Local: Sala Gl√≥ria Rocha do¬†Centro das Artes 
Data: 29/09/2024 
Faltam 15 dias para o evento

Evento: Conex√£o Jundia√≠-Joinville 
Local: Teatro Polytheama 
Data: 29/09/2024 
Faltam 15 dias para o evento

Evento: 8¬∫ FESTEJU – “Naquela Vila ou o Sumi√ßo de Epaminondas”, com o Grupo de Ter√ßa – Pracinha da Cultura do Vista Alegre 
Local: Sala Gl√≥ria Rocha do¬†Centro das Artes 
Data: 01/10/2024 
Faltam 17 dias para o evento

Evento: 8¬∫ FESTEJU – “Balb√∫rdia”, com a Companhia Arteenager 
Local: Sala Gl√≥ria Rocha do¬†Centro das Artes 
Data: 02/10/2024 
Faltam 18 dias para o evento

Evento: 8¬∫ FESTEJU – “Manifesto √† Beleza”, com o N√∫cleo Atuar√° 
Local: Sala Gl√≥ria Rocha do¬†Centro das Artes 
Data: 03/10/2024 
Faltam 19 dias para o event

ConnectionError: HTTPSConnectionPool(host='cultura.jundiai.sp.gov.br', port=443): Max retries exceeded with url: /agenda-cultural/page/9/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ef78b76f580>: Failed to establish a new connection: [Errno 113] No route to host'))